In [3]:
#Necessary for BeautifulSoup 
import sys
!{sys.executable} -m pip install geocoder

     |████████████████████████████████| 102kB 7.1MB/s ta 0:00:011
     |████████████████████████████████| 829kB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 15.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 28.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes 
import geopy.geocoders # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries are imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

### Getting data of Toronto

In [2]:
# Data from internet using wikipedia URL
wikipedia_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page= requests.get(wikipedia_url)
 

# Use beautiful soup to parse codes
soup = BeautifulSoup(wikipedia_page.content, 'html5lib')
table = soup.find('div', attrs = {'id':'container'})


In [3]:
#Extracting raw table

table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# extracting a clean form of the table

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [4]:
#Integrating Postal codes with more than 1 neighbour
unique_p = set(Postcode)
print('quantity of unique Postal codes:', len(unique_p))
Postcode_uni      = []
Borough_uni       = []
Neighbourhood_uni = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_uni.append(p_var)
    Borough_uni.append(b_var)
    Neighbourhood_uni.append(n_var)

quantity of unique Postal codes: 78


In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [6]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [7]:
#Installing geocoder
import geocoder

In [8]:
# initialize  variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286
5,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
6,M9A,Etobicoke,Islington Avenue,43.662299,-79.528195
7,M1B,Scarborough,Rouge,43.811525,-79.195517
8,M1B,Scarborough,Malvern,43.811525,-79.195517
9,M3B,North York,Don Mills North\n,43.749055,-79.362227


In [20]:
neighbors.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286


In [21]:
df_toronto = pd.DataFrame.from_dict(neighbors)
df_toronto.to_csv('toronto_postal_codes.csv')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286


### Create a map of Toronto using obtained data

In [22]:
# latitude and longtitude are extracted of google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers 
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### According to the map, I decide focusing on the "Downtown" Borough in Toronto

In [23]:
# select neighborhoods regarding to "Downtown" borough.
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto']
downtown_data.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
5,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
12,M5B,Downtown Toronto,Ryerson\n,43.657363,-79.378180
13,M5B,Downtown Toronto,Garden District\n,43.657363,-79.378180
26,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481


### Create a Map of Downton Toronto and Its Neighbourhoods

In [24]:
address_down = 'Downtown Toronto, Toronto'
latitude_down = 43.651070 
longitude_down = -79.347015
print('The geograpical coordinate of "Downtown Toronto" are: {}, {}.'.format(latitude_down, longitude_down))

map_Downtown = folium.Map(location=[latitude_down, longitude_down], zoom_start=11.5)

# add markers 
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Downtown)  
    
map_Downtown

The geograpical coordinate of "Downtown Toronto" are: 43.65107, -79.347015.


In [49]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;
    

In [42]:
foursquare_crawler

<function __main__.foursquare_crawler(postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT=500, radius=1000)>

In [43]:
# Foursquare Credentials

CLIENT_ID = 'NNOX1POZUNQFBDL4QXQBKIX2JW35ECWN5NWJPWIZKGWNNYJ0' # your Foursquare ID
CLIENT_SECRET = 'VJWJR5J12PV02QWPUJMTQMTCSPYYTAZ5NLJRT02GEY1RVCVJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NNOX1POZUNQFBDL4QXQBKIX2JW35ECWN5NWJPWIZKGWNNYJ0
CLIENT_SECRET:VJWJR5J12PV02QWPUJMTQMTCSPYYTAZ5NLJRT02GEY1RVCVJ


### Venues in the Neighborhoods inside "Downtown"

In [31]:
downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
5,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715
12,M5B,Downtown Toronto,Ryerson\n,43.657363,-79.378180
13,M5B,Downtown Toronto,Garden District\n,43.657363,-79.378180
26,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481


In [51]:
print('Neighborhoods inside "Downtown"')
Downtown_foursquare_dataset = foursquare_crawler(list(downtown_data['PostalCode']),
                                                   list(downtown_data['Neighborhood']),
                                                   list(downtown_data['Latitude']),
                                                   list(downtown_data['Longitude']),)

print('finalizado')

Neighborhoods inside "Downtown"
1.
Data is Obtained, for the Postal Code M5A (and Neighborhoods Harbourfront) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M7A (and Neighborhoods Queen's Park) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M5B (and Neighborhoods Ryerson
) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M5B (and Neighborhoods Garden District
) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M5C (and Neighborhoods St. James Town) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M5E (and Neighborhoods Berczy Park) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M5G (and Neighborhoods Central Bay Street
) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M6G (and Neighborhoods Christie
) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M5H (and Neighborhoods Adelaide
) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Code M5H (and Neighborhoods King
) SUCCESSFULLY.
11.
Data is Obtained, for the Postal Code M5H (and N

### Filtering Data Received from Foursquare Database

In [59]:
# Extract each venue for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
   
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Postal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
                      
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
           
    
    return(result_df)

In [60]:
downtown_venues = get_venue_dataset(Downtown_foursquare_dataset)

Number of Venuse in Coordination "M5A" Postal Code and "Harbourfront" Negihborhood(s) is:
97
Number of Venuse in Coordination "M7A" Postal Code and "Queen's Park" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5B" Postal Code and "Ryerson
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5B" Postal Code and "Garden District
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5C" Postal Code and "St. James Town" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5E" Postal Code and "Berczy Park" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5G" Postal Code and "Central Bay Street
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M6G" Postal Code and "Christie
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5H" Postal Code and "Adelaide
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5H" Postal Code and "King
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5H" Postal Code and "Richmond
"

### Venues for Each Neighborhood in Downtown

In [61]:
downtown_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M5A,Harbourfront,43.650295,-79.359166,The Distillery Historic District,This spot is popular,Historic Site,13
1,M5A,Harbourfront,43.650295,-79.359166,Distillery Sunday Market,This spot is popular,Farmers Market,216
2,M5A,Harbourfront,43.650295,-79.359166,Arvo,This spot is popular,Coffee Shop,187
3,M5A,Harbourfront,43.650295,-79.359166,Cacao 70,This spot is popular,Dessert Shop,127
4,M5A,Harbourfront,43.650295,-79.359166,SOMA chocolatemaker,This spot is popular,Chocolate Shop,91


In [62]:
downtown_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
3213,M4Y,Church and Wellesley,43.666585,-79.381302,Starbucks,This spot is popular,Coffee Shop,699
3214,M4Y,Church and Wellesley,43.666585,-79.381302,Okonomi House お好みハウス,This spot is popular,Japanese Restaurant,495
3215,M4Y,Church and Wellesley,43.666585,-79.381302,GoodLife Fitness,This spot is popular,Gym,591
3216,M4Y,Church and Wellesley,43.666585,-79.381302,The Captain's Boil,This spot is popular,Seafood Restaurant,501
3217,M4Y,Church and Wellesley,43.666585,-79.381302,Bulk Barn,This spot is popular,Grocery Store,594


In [63]:
# Save a dataFrame with the Results from Foursquare
downtown_venues.to_csv('downtown_venues.csv')

In [2]:
# Use data obtained from foursquare
downtown_venues = pd.read_csv('downtown_venues.csv')

In [65]:
# Results from foursquare
neighb_lst = list(downtown_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Downtown:')
print(len(neighb_lst))
print('Neighborhoods inside Downtown:')
neighb_lst

Number of Neighborhoods inside Downtown:
36
Neighborhoods inside Downtown:


['Harbourfront',
 "Queen's Park",
 'Ryerson\n',
 'Garden District\n',
 'St. James Town',
 'Berczy Park',
 'Central Bay Street\n',
 'Christie\n',
 'Adelaide\n',
 'King\n',
 'Richmond\n',
 'Harbourfront East\n',
 'Toronto Islands',
 'Union Station',
 'Design Exchange',
 'Toronto Dominion Centre',
 'Commerce Court',
 'Victoria Hotel\n',
 'Harbord\n',
 'University of Toronto',
 'Chinatown',
 'Grange Park',
 'Kensington Market',
 'CN Tower',
 'Bathurst Quay\n',
 'Island airport\n',
 'Harbourfront West\n',
 'King and Spadina',
 'Railway Lands',
 'South Niagara',
 'Rosedale',
 'Stn A PO Boxes 25 The Esplanade\n',
 'Cabbagetown',
 'First Canadian Place',
 'Underground city',
 'Church and Wellesley']

In [71]:
neighb_venue_summary = downtown_venues.groupby('Neighborhood').count()
neighb_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Adelaide\n,100,100,100,100,100,100,100
Bathurst Quay\n,100,100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100,100
CN Tower,100,100,100,100,100,100,100
Cabbagetown,38,38,38,38,38,38,38


### Summary of venues

In [81]:
# Summary
downtown_onehot = pd.get_dummies(data = downtown_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
downtown_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Afghan Restaurant,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,M5A,Harbourfront,43.650295,-79.359166,The Distillery Historic District,This spot is popular,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M5A,Harbourfront,43.650295,-79.359166,Distillery Sunday Market,This spot is popular,216,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Filter Restaurant features

In [8]:
important_list_of_features = ['Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'American Restaurant',
 'Asian Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Greek Restaurant',
 'Grocery Store',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant']                          
                              
                              

In [12]:
important_list_of_features

['Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'American Restaurant',
 'Asian Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Greek Restaurant',
 'Grocery Store',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant']

In [82]:
downtown_onehot = downtown_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Longitude']).groupby(
    'Neighborhood Latitude').sum()

downtown_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Filipino Restaurant,Greek Restaurant,Grocery Store,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
Neighborhood Latitude,,,,,,,,,,,,,,,,,,,,,,,,
43.630210,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43.640815,0,7,7,14,0,7,0,0,0,0,0,35,7,0,7,7,7,0,7,7,0,7,7,0
43.645160,1,2,0,0,0,0,0,0,1,0,0,1,5,0,0,0,0,1,1,0,0,1,1,0
43.647100,0,4,0,0,0,2,0,2,0,0,0,8,6,0,0,2,0,0,4,2,0,4,4,0
43.648280,2,4,2,0,0,2,0,2,0,0,0,6,8,0,2,2,0,0,4,2,0,4,4,0


### Agrupando restaurantes sesgun barrio

In [83]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 4, random_state = 0).fit(downtown_onehot)

In [84]:
me_df = pd.DataFrame(kmeans.cluster_centers_)
me_df.columns = downtown_onehot.columns
me_df.index = ['Kl1','Kl2','KL3','KL4']
me_df['Total Sum'] = me_df.sum(axis = 1)
me_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Neighborhood,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Filipino Restaurant,Greek Restaurant,Grocery Store,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Total Sum
Kl2,0.000000,7.000000,7.000000,14.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,35.0,7.000000,0.0,7.000000,7.000000,7.000000,0.000000,7.000000,7.000000,0.000000,7.000000,7.000000,0.000000,126.000000
KL3,0.000000,0.000000,0.000000,3.000000,6.000000,0.000000,3.000000,0.000000,6.000000,3.000000e+00,0.000000,0.0,3.000000,0.0,0.000000,0.000000,9.000000,0.000000,0.000000,3.000000,0.000000,3.000000,18.000000,12.000000,69.000000
KL4,1.250000,4.500000,1.250000,0.000000,0.000000,1.750000,0.000000,2.250000,0.000000,0.000000e+00,0.000000,5.5,9.250000,0.0,1.750000,1.750000,0.000000,0.000000,5.000000,1.750000,0.000000,3.250000,5.250000,0.000000,44.500000
Kl1,0.538462,1.230769,0.230769,0.615385,0.384615,0.230769,0.153846,0.307692,1.153846,-2.775558e-17,0.461538,2.0,3.076923,1.0,0.307692,0.230769,1.230769,0.923077,0.846154,1.076923,0.153846,1.230769,1.461538,0.230769,19.076923


In [88]:
neighborhood_summary = pd.DataFrame([downtown_onehot.index, 1 + kmeans.labels_]).T
neighborhood_summary.columns = ['Neighborhood Latitude', 'Group']
neighborhood_summary

,Neighborhood Latitude,Group
0,43.630210,1.0
1,43.640815,2.0
2,43.645160,1.0
3,43.647100,4.0
4,43.648280,4.0
5,43.648395,4.0
6,43.648690,1.0
7,43.649700,4.0
8,43.650295,1.0
9,43.651210,1.0


### Selecting the best Neighborhood

In [89]:
#Cluster number 1
neighborhood_summary[neighborhood_summary['Group'] == 2]

,Neighborhood Latitude,Group
1,43.640815,2.0


In [99]:
name_of_neigh = list(neighborhood_summary[neighborhood_summary['Group'] == 2]['Neighborhood Latitude'])[0]
#name_of_neigh
downtown_venues[downtown_venues['Neighborhood Latitude'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M5V',
 'Neighborhood': 'CN Tower',
 'Neighborhood Latitude': 43.64081500000003,
 'Neighborhood Longitude': -79.39953781899999}

In [101]:
#number 2

neighborhood_summary[neighborhood_summary['Group'] == 3]

,Neighborhood Latitude,Group
10,43.65353,3.0


In [102]:
name_of_neigh = list(neighborhood_summary[neighborhood_summary['Group'] == 3]['Neighborhood Latitude'])[0]
#name_of_neigh
downtown_venues[downtown_venues['Neighborhood Latitude'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M5T',
 'Neighborhood': 'Chinatown',
 'Neighborhood Latitude': 43.653530000000046,
 'Neighborhood Longitude': -79.39723268299997}

In [103]:
#number 3

neighborhood_summary[neighborhood_summary['Group'] == 4]

,Neighborhood Latitude,Group
3,43.647100,4.0
4,43.648280,4.0
5,43.648395,4.0
7,43.649700,4.0


In [104]:
name_of_neigh = list(neighborhood_summary[neighborhood_summary['Group'] == 4]['Neighborhood Latitude'])[0]
#name_of_neigh
downtown_venues[downtown_venues['Neighborhood Latitude'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M5K',
 'Neighborhood': 'Design Exchange',
 'Neighborhood Latitude': 43.64710000000007,
 'Neighborhood Longitude': -79.38153109899997}

In [105]:
# number 4
neighborhood_summary[neighborhood_summary['Group'] == 1]

,Neighborhood Latitude,Group
0,43.630210,1.0
2,43.645160,1.0
6,43.648690,1.0
8,43.650295,1.0
9,43.651210,1.0
11,43.656091,1.0
12,43.657363,1.0
13,43.661150,1.0
14,43.663110,1.0
15,43.666585,1.0


In [106]:
name_of_neigh = list(neighborhood_summary[neighborhood_summary['Group'] == 1]['Neighborhood Latitude'])[0]
#name_of_neigh
downtown_venues[downtown_venues['Neighborhood Latitude'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M5J',
 'Neighborhood': 'Harbourfront East\n',
 'Neighborhood Latitude': 43.63021000000003,
 'Neighborhood Longitude': -79.36243320899995}